# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,8.95,73,65,7.70,CL,1699575245
1,1,miles city,46.4083,-105.8406,3.54,60,0,2.57,US,1699575189
2,2,adamstown,-25.0660,-130.1015,22.73,86,100,6.72,PN,1699575247
3,3,vilyuchinsk,52.9306,158.4028,-0.85,85,89,2.00,RU,1699574976
4,4,santana,-0.0583,-51.1817,30.10,74,40,6.17,BR,1699574768


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"    
)

# Display the map plot
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
filtered = city_data_df[(city_data_df["Humidity"] < 40) & (city_data_df["Max Temp"] > 15)]

# Drop any rows with null values
filtered = filtered.dropna()

# Display sample data
filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,puerto penasco,31.3167,-113.5333,23.84,28,0,4.02,MX,1699575259
52,52,thaba nchu,-29.2093,26.8390,15.73,35,0,2.80,ZA,1699575296
64,64,ghadamis,30.1337,9.5007,22.41,23,85,5.05,LY,1699574897
71,71,altar,30.7000,-111.8167,22.24,17,0,4.37,MX,1699575323
84,84,benjamin hill,30.1667,-111.1000,20.88,19,0,6.09,MX,1699575341
116,116,alice springs,-23.7000,133.8833,29.75,32,49,4.63,AU,1699575264
123,123,afif,23.9065,42.9172,18.75,35,10,4.61,SA,1699575387
148,148,san agustin de valle fertil,-30.6335,-67.4682,21.32,37,38,4.92,AR,1699575414
191,191,tamanrasset,22.7850,5.5228,16.95,15,0,2.06,DZ,1699575458
234,234,goure,13.9835,10.2704,25.77,17,76,3.36,NE,1699575504


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Create a copy of the original DataFrame with selected columns
hotel_df = filtered.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty "Hotel Name" column
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
116,alice springs,AU,-23.7000,133.8833,32,
202,maryborough,AU,-25.5333,152.7000,46,
234,goure,NE,13.9835,10.2704,17,
247,posto fiscal rolim de moura,BR,-13.0833,-62.2667,46,
272,lospalos,TL,-8.5186,127.0025,39,
284,kone,NC,-21.0595,164.8658,46,
314,shakawe,BW,-18.3667,21.8500,26,
322,santa ana,US,33.7456,-117.8678,17,
356,edd,ER,13.9297,41.6925,44,
399,kidal,ML,18.4411,1.4078,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set the parameters for the type of search
limit = 1
radius = 10000
categories = "lodging"
conditions = "hotel"

# set up a parameters dictionary
params = {
    "radius":radius,
    "categories":categories,
    "conditions":conditions,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alice springs - nearest hotel: No hotel found
maryborough - nearest hotel: No hotel found
goure - nearest hotel: No hotel found
posto fiscal rolim de moura - nearest hotel: No hotel found
lospalos - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
santa ana - nearest hotel: No hotel found
edd - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
charters towers - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
aldinga beach - nearest hotel: No hotel found
gwadar - nearest hotel: No hotel found
arraias - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
116,alice springs,AU,-23.7000,133.8833,32,No hotel found
202,maryborough,AU,-25.5333,152.7000,46,No hotel found
234,goure,NE,13.9835,10.2704,17,No hotel found
247,posto fiscal rolim de moura,BR,-13.0833,-62.2667,46,No hotel found
272,lospalos,TL,-8.5186,127.0025,39,No hotel found
284,kone,NC,-21.0595,164.8658,46,No hotel found
314,shakawe,BW,-18.3667,21.8500,26,No hotel found
322,santa ana,US,33.7456,-117.8678,17,No hotel found
356,edd,ER,13.9297,41.6925,44,No hotel found
399,kidal,ML,18.4411,1.4078,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map
hover_cols = ["City", "Country", "Hotel Name"]

map_plot1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=hover_cols    
)

# Display the map plot
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)